Для начала импортируем необходимые библиотеки:

In [ ]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import metrics #метрики
from sklearn import model_selection #методы разделения и валидации
from sklearn import ensemble #ансамбли

plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
%matplotlib inline

1. Прочитайте таблицу с данными ('data/online_shoppers_intention.csv') и выведите ее на экран, чтобы убедиться, что чтение прошло успешно.

In [ ]:
df = pd.read_csv('data/online_shoppers_intention.csv')
print(df.head())

2. Выведите размер таблицы

In [ ]:
print(f"Строк {df.shape[0]}. Столбцов {df.shape[1]}")
assert df.shape[0] > 12000 and df.shape[1] == 18

3. В нашей таблице содержится информация о более чем 12 тысячах сессий на сайте интернет-магазина. Каждая сессия описывается 18 признаками.
Удостоверьтесь в отсутствии пропусков:

In [ ]:
nulls = df.isnull().sum()
print(nulls)
for col, sum in nulls:
    assert sum!=0, f"Имеются пропуски в колонке {col}"

4. Закодируйте категориальные признаки с помощью простого горячего кодирования, используя уже знакомую нам функцию get_dummies():

In [ ]:
types = df.dtypes
#Категориальные признаки
cat_features = list(types[(types == 'object')].index)
print(cat_features) # ['Month', 'VisitorType']
n = len(cat_features) #число категориальных признаков
fig, axes = plt.subplots(1, 2, figsize=(40, 20))
#Строим столбчатую диаграмму для категории Месяц
sns.barplot(data=df, x=cat_features[0], y='Revenue', ax=axes[0])
#Строим столбчатую диаграмму для категории Тип посетителя
sns.barplot(data=df, x=cat_features[1], y='Revenue', ax=axes[1])

plt.tight_layout()
plt.show()
#по столбчатым диаграммам видим что максимум приходится на конец года - Октябрь, Ноябрь, Декабь
# в Январе и феврале - минимум продаж - этот признак отбрасывать нельзя

#по типу клиента связь выражена не так сильно как по месяцам, но все-же есть. (тоже не отбрасываем)

5. Теперь, когда необходимые преобразования выполнены, мы можем говорить о построении модели.

Итак, нам необходимо предсказать целевую переменную Revenue — признак покупки. Целевой признак является бинарным категориальным, то есть мы решаем задачу бинарной классификации.
В первую очередь визуализируйте соотношение классов в данных:

In [ ]:
sns.countplot(data=df, x='Revenue')
plt.show()


6. Сбалансирована ли данная выборка? Обоснуйте свою позицию

In [ ]:
Выборка не сбалансирована. Купленных машин в несколько раз меньше по завершении сессии.

7. Из 12330 сессий покупкой товара завершаются лишь 15.47 %. Мы знаем, что такое соотношение классов заставляет нас смотреть на метрики для каждого из классов отдельно.

Условимся, что лучшей будет считаться та модель, у которой значение метрики F1 для пользователей, совершивших покупку, будет наибольшим.

Разделите набор данных на матрицу наблюдений X и вектор ответов y:

Давайте заранее определимся, как мы будем производить контроль качества наших моделей:

Разделим выборку на тренировочную и тестовую.
Будем проводить кросс-валидацию на тренировочной выборке (то есть будем делить её на тренировочные и валидационные фолды и считать среднее значение метрики по фолдам).
Итого мы будем использовать два показателя:

значение метрики на тренировочных и валидационных фолдах кросс-валидации (по ним мы будем отслеживать переобучение модели и подбирать внешние параметры);
значение метрики на отложенной тестовой выборке (оно будет нашим контрольным показателем).
Другими словами, мы будем сочетать hold-оut- и k-fold-подходы к валидации.

8. Для начала позаботимся о создании отложенной тестовой выборки.

Разделите выборку на тренировочную и тестовую в соотношении 80/20. Используйте разбиение, стратифицированное по целевому признаку. В качестве значения параметра random_state возьмите число 42.

Чему равно количество сессий на сайте в тренировочной выборке?

9. Расчитайте количество сессий в тестовой выборке:

10.Коллеги посоветовали нам использовать случайный лес (Random Forest) для решения данной задачи. Давайте последуем их совету.

Создайте модель случайного леса. В качестве значения параметра random_state возьмите число 42. Остальные параметры оставьте по умолчанию.

Оцените качество такой модели с помощью кросс-валидации по пяти фолдам. Так как классы несбалансированы, используйте кросс-валидатор StratifiedKFold (перемешивать выборку не нужно).

Для проведения кросс-валидации используйте функцию cross_validate(). Набор данных (параметры X, y) — тренировочная выборка (X_train, y_train). Метрика — F1-score.

Расчитайте, чему равно среднее значение метрики  на тренировочных и валидационных фолдах?

11. Является ли, по-вашему, построенная в предыдущем задании модель случайного леса переобученной? Обоснуйте вашу позицию

12. Попробуем использовать несколько вариаций случайного леса и с помощью кривых обучения постараемся выбрать наилучшую из них.

Создайте список из трёх следующих моделей:

Случайный лес из деревьев максимальной глубины 5.
Случайный лес из деревьев максимальной глубины 7.
Случайный лес из деревьев максимальной глубины 12.
Для всех трёх моделей количество деревьев в лесу (n_estimators) возьмите равным 200, количество объектов в листе (min_samples_leaf) — 5. Параметр random_state = 42. Остальные параметры оставьте по умолчанию.

Постройте для каждой из моделей кривую обучения.
Совет: воспользуйтесь функцией plot_learning_curve()

Для построения кривых используйте обучающий набор данных (X_train, y_train), стратифицированный кросс-валидатор на пяти фолдах (StratifiedKFold) и метрику F1-score. Остальные параметры функции learning_curve() оставьте по умолчанию.

13. Из построенных кривых обучения сделайте вывод: какая глубина деревьев в лесу является оптимальной? Ответ обоснуйте

14. Обучите случайный лес с выбранной в предыдущем задании оптимальной глубиной на тренировочной выборке. Сделайте предсказание меток классов и выведите отчёт о метриках классификации.

 Рассчитайте значение метрики accuracy

15. Рассчитайте значение метрики F1 для посетителей, завершивших сессию без покупки товара?

16. Рассчитайте значение метрики F1 для посетителей, купивших товар во время сессии?

17. Напишите вывод по полученным значениям

18. Попробуем повысить качество распознавания посетителей, совершивших покупку. Используем метод подбора порога вероятности с помощью PR-кривой.

Порог вероятности будем подбирать с помощью кросс-валидации.

Сделайте предсказание вероятностей принадлежности к пользователям, которые совершат покупку, на кросс-валидации на пяти фолдах. Используйте метод cross_val_predict().

Для кросс-валидации используйте случайный лес с подобранной в прошлых заданиях оптимальной максимальной глубиной деревьев, набор данных (параметры X, y) — тренировочная выборка (X_train, y_train).

Постройте PR-кривую и отметьте на ней точку, в которой наблюдается максимум метрики  для посетителей, которые совершат покупку. Определите порог вероятности, соответствующий этой точке.

19. Сделайте предсказание классов объекта с определённым в предыдущем задании порогом вероятности. Выведите отчёт о метриках классификации.
Рассчитайте значение метрики accuracy

20. Рассчитайте значение метрики F1 для посетителей, завершивших сессию без покупки товара и значение метрики F1 для посетителей, купивших товар во время сессии? После расчета напишите свои выводы на основе метрик